In [312]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [311]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import html5lib
import re
from scipy.special import erf
import math
from ipywidgets import widgets, Layout
from IPython.display import display


In [317]:
def repull_data(b):
    url='http://www.kenpom.com/'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')

    table = soup.find(id='ratings-table')
    for body in table("tbody"):
        body.unwrap()
    for body in table("thead"):
        body.unwrap()
    columns = ['Rank','Team','Conf','W-L','AdjEM','AdjO','AdjO_rank','AdjD',
           'AdjD_rank','AdjT','AdjT_rank','Luck','Luck_rank','AdjEM_SoS','AdjEM_SoS_rank',
           'OppO_SoS','OppO_SoS_rank','OppD_SoS','OppD_SoS_rank','AdjEM_NCSOS','AdjEM_NCSOS_rank']
    kp_data = pd.read_html(str(table), flavor='bs4')
    # data = data[data['Rk'] != 'Rk']
    data = kp_data[0]
    data.columns = columns
    data = data.loc[2:len(data)]
    data.loc[:,'Team'] = data.loc[:,'Team'].apply(lambda x: re.sub('[0-9]','',x))
    team_list = [str(y) for y in list(data['Team'])]
    team_list.sort()

def calc_point_diff(team1, team2):
    AdjEM_A = float(data[data['Team']==team1]['AdjEM'])
    AdjEM_B = float(data[data['Team']==team2]['AdjEM'])
    AdjT_A = float(data[data['Team']==team1]['AdjT'])
    AdjT_B = float(data[data['Team']==team2]['AdjT'])
    return (AdjEM_A - AdjEM_B)*(AdjT_A + AdjT_B)/200

def win_percent(point_diff):
    x = 0
    sigma=11
    return 0.5 * (1 + erf((x - point_diff)/(sigma*math.sqrt(2))))

def handle_submit(sender):
    team1_name = team_1.value
    team2_name = team_2.value
    point_diff = calc_point_diff(team1_name, team2_name)
    win_pct = 100.0 * win_percent(point_diff)
    html_string = """{0} vs {1}<br>Point Diff: {2}<br>{0} win pct: {3}%<br>{1} win pct: {4}%"""
    result.value = html_string.format(
            team1_name, team2_name, point_diff, round(100.0-win_pct,2),round(win_pct,2))


In [318]:
repull_data(None)

In [319]:
team_1 = widgets.Select(
    options=team_list,
    description='',
    disabled=False
)

team_2 = widgets.Select(
    options=team_list,
    description='',
    disabled=False
)

result = widgets.HTML(
    value="",
    placeholder='Some HTML',
    description='Some HTML',
    disabled=False
)

update_data = widgets.Button(description="Update the data")


team_1.observe(handle_submit)
team_2.observe(handle_submit)
update_data.on_click(repull_data)

selectors = widgets.HBox([team_1,result,team_2], layout=Layout(justify_content='space-between'))
buttons = widgets.HBox([update_data], layout=Layout(justify_content='center'))
everything = widgets.VBox([buttons,selectors], layout=Layout(justify_content='space-between'))

In [320]:
display(everything)